# NYT counties data ingress

Note:  this notebook initializes the database with the time series from The New York Times counties database.


In [ ]:
import os

In [ ]:
import tqdm

In [ ]:
from covidvu.cryostation import Cryostation

In [ ]:
from covidvu.config import MASTER_DATABASE

In [ ]:
import csv
import shutil

In [ ]:
databasePath = os.path.join('..', MASTER_DATABASE)
databaseBackupPath = databasePath.replace('.db', '_db.bak')

In [ ]:
NYT_COUNTY_DATA='../NYT-data-repository/us-counties.csv'

In [ ]:
def backupDatabase():
    shutil.copy(databasePath, databaseBackupPath)

In [ ]:
def restoreDatabase():
    shutil.copy(databaseBackupPath, databasePath)

## Backups and get ready

In [ ]:
backupDatabase()

In [ ]:
with Cryostation(databasePath) as c:
    county = c['US']['provinces']['California']['counties']['San Francisco']

county

## Extract the counties and states

In [ ]:
dataset = {}

In [ ]:
with open(NYT_COUNTY_DATA) as inputFile:
    reader = csv.DictReader(inputFile)
    
    for element in tqdm.tqdm(reader):
        state = element['state']
        if state not in dataset:
            dataset[state] = { 'counties': dict() }
        
        county    = element['county']
        date      = element['date']
        confirmed = float(element['cases'])
        deaths    = float(element['deaths'])
        
        if county not in dataset[state]['counties']:
            dataset[state]['counties'][county] = { 'confirmed': dict(), 'deaths': dict(), }
        
        dataset[state]['counties'][county]['confirmed'][date] = confirmed
        dataset[state]['counties'][county]['deaths'][date] = deaths

## Apply counties data to each state

In [ ]:
with Cryostation(databasePath) as cryostation:
    country = cryostation['US']

In [ ]:
for state in tqdm.tqdm(country['provinces']):
    if 'counties' not in country['provinces'][state]:
        continue
    
    for county in country['provinces'][state]['counties']:
        if county not in dataset[state]['counties']:
            continue
        country['provinces'][state]['counties'][county]['confirmed'] = dataset[state]['counties'][county]['confirmed']
        country['provinces'][state]['counties'][county]['deaths'] = dataset[state]['counties'][county]['deaths']

## Update the database and done

In [ ]:
with Cryostation(databasePath) as cryostation:
    cryostation['US'] = country

In [ ]:
country['provinces']['California']['counties']['San Francisco']

## Read the lastest and display a county

In [ ]:
with Cryostation(databasePath) as c:
    county = c['US']['provinces']['California']['counties']['San Francisco']

In [ ]:
county

In [ ]:
restoreDatabase()